In [1]:
from quantumgraph import ExpectationValue

An IBMQ account could not be loaded


In [2]:
import numpy as np
import time
from qiskit import QuantumCircuit

In [3]:
# are single qubit operations okay?
n = 2
qc = QuantumCircuit(n)
qc.h(0)
qc.ry(-np.pi/8,1)
ev = ExpectationValue(n)
ev.apply_circuit(qc)
assert ev.pauli_decomp['XI']==1, ev.pauli_decomp
assert ev.pauli_decomp['XX']==-0.3826834323650898, ev.pauli_decomp
assert ev.pauli_decomp['XZ']==.9238795325112867, ev.pauli_decomp

# does a cx copy an x?
n = 2
qc = QuantumCircuit(n)
qc.x(0)
qc.cx(0,1)
ev = ExpectationValue(n)
ev.apply_circuit(qc)
assert ev.pauli_decomp['IZ']==-1, ev.pauli_decomp

# are all the 2 qubit correlators present in a 3 qubit ghz state?
n = 3
qc = QuantumCircuit(n)
qc.h(0)
qc.cx(0,1)
qc.cx(1,2)
ev = ExpectationValue(n)
ev.apply_circuit(qc)
assert ev.pauli_decomp['ZZI']==1, ev.pauli_decomp
assert ev.pauli_decomp['ZIZ']==1, ev.pauli_decomp
assert ev.pauli_decomp['IZZ']==1, ev.pauli_decomp

# is everything present when using k=3?
ev = ExpectationValue(n,k=3)
ev.apply_circuit(qc)
assert ev.pauli_decomp['ZZI']==1, ev.pauli_decomp
assert ev.pauli_decomp['ZIZ']==1, ev.pauli_decomp
assert ev.pauli_decomp['IZZ']==1, ev.pauli_decomp
assert ev.pauli_decomp['XYY']==-1, ev.pauli_decomp
assert ev.pauli_decomp['YXY']==-1, ev.pauli_decomp
assert ev.pauli_decomp['YYX']==-1, ev.pauli_decomp
assert ev.pauli_decomp['XXX']==1, ev.pauli_decomp

# does it run  fast for 50 qubits? (k=2 and no pairs)
num_qubits = 50
qc = QuantumCircuit(num_qubits)
qc.h(0)
for j in range(1,num_qubits):
    qc.ry(np.pi/10,j)
    qc.cx(j-1,j)
t0 = time.time()
ev = ExpectationValue(num_qubits)
ev.apply_circuit(qc)
t = time.time()-t0
assert t<10, t

# does it run  fast for 50 qubits? (k=3 and nearest neighbours)
pairs = []
for j in range(num_qubits-4):
    for delta in range(1,5):
        pairs.append([j,j+delta])
t0 = time.time()
ev = ExpectationValue(num_qubits,k=3,pairs=pairs)
ev.apply_circuit(qc)
t = time.time()-t0
assert t<10, t